# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse  Shipments          Customer  \
0  Jun 15 2022 12:00AM  239603        10  MET211245  Methapharm, Inc.   
1  Jun 15 2022 12:00AM  239603        10  MET211255  Methapharm, Inc.   
2  Jun 15 2022 12:00AM  239603        10  MET211257  Methapharm, Inc.   
3  Jun 15 2022 12:00AM  239603        10  MET211258  Methapharm, Inc.   
4  Jun 15 2022 12:00AM  239604        10  MET211259  Methapharm, Inc.   
5  Jun 15 2022 12:00AM  239603        10  MET211262  Methapharm, Inc.   
6  Jun 15 2022 12:00AM  239603        10  MET211263  Methapharm, Inc.   
7  Jun 15 2022 12:00AM  239603        10  MET211264  Methapharm, Inc.   
8  Jun 15 2022 12:00AM  239603        10  MET211265  Methapharm, Inc.   
9  Jun 15 2022 12:00AM  239603        10  MET211266  Methapharm, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
39  239600       Released          1
40  239601       Released          1
41  239602       Released          1
42  239603       Released         22
43  239604       Released          4

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
239600                NaN        NaN       1.0
239601                NaN        NaN       1.0
239602                NaN        NaN       1.0
239603                NaN        NaN      22.0
239604                NaN        NaN       4.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
239446               74.0        0.0      79.0
239525                0.0        0.0       1.0
239526                9.0        2.0       3.0
239533               20.0        2.0       0.0
239535               11.0        3.0      16.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
239446                 74          0        79
239525                  0          0         1
239526                  9          2         3
239533                 20          2         0
239535                 11          3        16

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               239446         74          0        79
1               239525          0          0         1
2               239526          9          2         3
3               239533         20          2         0
4               239535         11          3        16

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               239446        74                 79
1               239525                            1
2               239526         9         2        3
3               239533        20         2         
4               239535        11         3       16

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                 Customer
0   Jun 15 2022 12:00AM  239603        10         Methapharm, Inc.
4   Jun 15 2022 12:00AM  239604        10         Methapharm, Inc.
26  Jun 15 2022 12:00AM  239601        10         Methapharm, Inc.
27  Jun 15 2022 12:00AM  239602        10         Methapharm, Inc.
28  Jun 15 2022 12:00AM  239599        10         Methapharm, Inc.
29  Jun 15 2022 12:00AM  239600        10         Methapharm, Inc.
30  Jun 14 2022  3:47PM  239595        21        NBTY Global, Inc.
31  Jun 14 2022  3:45PM  239593        21        NBTY Global, Inc.
32  Jun 14 2022  3:30PM  239591        19      AdvaGen Pharma, Ltd
59  Jun 14 2022  3:27PM  239590        19  GUSA Granules USA, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                 Customer Completed  \
0  Jun 15 2022 12:00AM  239603        10         Methapharm, Inc.             
1  Jun 15 2022 12:00AM  239604        10         Methapharm, Inc.             
2  Jun 15 2022 12:00AM  239601        10         Methapharm, Inc.             
3  Jun 15 2022 12:00AM  239602        10         Methapharm, Inc.             
4  Jun 15 2022 12:00AM  239599        10         Methapharm, Inc.             
5  Jun 15 2022 12:00AM  239600        10         Methapharm, Inc.             
6  Jun 14 2022  3:47PM  239595        21        NBTY Global, Inc.             
7  Jun 14 2022  3:45PM  239593        21        NBTY Global, Inc.             
8  Jun 14 2022  3:30PM  239591        19      AdvaGen Pharma, Ltd             
9  Jun 14 2022  3:27PM  239590        19  GUSA Granules USA, Inc.             

  Executing Released  
0                 22  
1                  4  
2                  1  
3                  1  
4                  1  
5                  1  
6                  1  
7                  1  
8         1       26  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                 Customer Released  \
0  Jun 15 2022 12:00AM  239603        10         Methapharm, Inc.       22   
1  Jun 15 2022 12:00AM  239604        10         Methapharm, Inc.        4   
2  Jun 15 2022 12:00AM  239601        10         Methapharm, Inc.        1   
3  Jun 15 2022 12:00AM  239602        10         Methapharm, Inc.        1   
4  Jun 15 2022 12:00AM  239599        10         Methapharm, Inc.        1   
5  Jun 15 2022 12:00AM  239600        10         Methapharm, Inc.        1   
6  Jun 14 2022  3:47PM  239595        21        NBTY Global, Inc.        1   
7  Jun 14 2022  3:45PM  239593        21        NBTY Global, Inc.        1   
8  Jun 14 2022  3:30PM  239591        19      AdvaGen Pharma, Ltd       26   
9  Jun 14 2022  3:27PM  239590        19  GUSA Granules USA, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8         1            
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse          Customer Released Executing  \
0  Jun 15 2022 12:00AM  239603        10  Methapharm, Inc.       22             
1  Jun 15 2022 12:00AM  239604        10  Methapharm, Inc.        4             
2  Jun 15 2022 12:00AM  239601        10  Methapharm, Inc.        1             
3  Jun 15 2022 12:00AM  239602        10  Methapharm, Inc.        1             
4  Jun 15 2022 12:00AM  239599        10  Methapharm, Inc.        1             

  Completed  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Jun 15 2022 12:00AM  239603        10  Methapharm, Inc.      22.0   
1  Jun 15 2022 12:00AM  239604        10  Methapharm, Inc.       4.0   
2  Jun 15 2022 12:00AM  239601        10  Methapharm, Inc.       1.0   
3  Jun 15 2022 12:00AM  239602        10  Methapharm, Inc.       1.0   
4  Jun 15 2022 12:00AM  239599        10  Methapharm, Inc.       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Jun 15 2022 12:00AM  239603        10  Methapharm, Inc.      22.0   
1  Jun 15 2022 12:00AM  239604        10  Methapharm, Inc.       4.0   
2  Jun 15 2022 12:00AM  239601        10  Methapharm, Inc.       1.0   
3  Jun 15 2022 12:00AM  239602        10  Methapharm, Inc.       1.0   
4  Jun 15 2022 12:00AM  239599        10  Methapharm, Inc.       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3354016      42.0        5.0       37.0
12          239544       0.0        2.0        0.0
15          239446      79.0        0.0       74.0
18         1197874       5.0        0.0        0.0
19         1916604      37.0        1.0        0.0
20          479086      19.0       27.0       12.0
21          718777       3.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3354016      42.0        5.0       37.0
1        12   239544       0.0        2.0        0.0
2        15   239446      79.0        0.0       74.0
3        18  1197874       5.0        0.0        0.0
4        19  1916604      37.0        1.0        0.0
5        20   479086      19.0       27.0       12.0
6        21   718777       3.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      42.0        5.0       37.0
1        12       0.0        2.0        0.0
2        15      79.0        0.0       74.0
3        18       5.0        0.0        0.0
4        19      37.0        1.0        0.0
5        20      19.0       27.0       12.0
6        21       3.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   42.0
1        12  Released    0.0
2        15  Released   79.0
3        18  Released    5.0
4        19  Released   37.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   18    19    20   21
Status                                          
Completed  37.0  0.0  74.0  0.0   0.0  12.0  0.0
Executing   5.0  2.0   0.0  0.0   1.0  27.0  0.0
Released   42.0  0.0  79.0  5.0  37.0  19.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   18    19    20   21
0          Completed  37.0  0.0  74.0  0.0   0.0  12.0  0.0
1          Executing   5.0  2.0   0.0  0.0   1.0  27.0  0.0
2           Released  42.0  0.0  79.0  5.0  37.0  19.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   18    19    20   21
0  Completed  37.0  0.0  74.0  0.0   0.0  12.0  0.0
1  Executing   5.0  2.0   0.0  0.0   1.0  27.0  0.0
2   Released  42.0  0.0  79.0  5.0  37.0  19.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()